In [43]:
data = pd.read_csv('test.tsv', sep='\t', names=['phrase', 'context', 'id'], skiprows=1)


In [45]:
import torch
import pandas as pd

from sbert_punc_case_ru import SbertPuncCase

device = torch.device('cuda:10')
model = SbertPuncCase().to(device)

data = pd.read_csv('test.tsv', sep='\t', names=['phrase', 'context', 'id'], skiprows=1)
data['context'] = data['context'].apply(lambda x: eval(x))
data['punc_phrase'] = data['phrase'].apply(lambda x: model.punctuate(x))

def punctuate_user_context(list_of_replics):
    new_list_of_replics = []
    for replic in list_of_replics:
        user_replic = replic['user']
        if user_replic:
            user_replic = model.punctuate(user_replic)
        new_list_of_replics.append({'user': user_replic, 'marusia': replic['marusia']})
    return new_list_of_replics

data['punc_context'] = data['context'].apply(lambda x: punctuate_user_context(x))

In [66]:
model.to('cpu')

SbertPuncCase(
  (model): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(120138, 1024, padding_idx=0)
        (position_embeddings): Embedding(512, 1024)
        (token_type_embeddings): Embedding(2, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_features=1024, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=1024, out_features=1024

In [48]:
data = data[['punc_phrase', 'punc_context', 'id']]
data.columns = ['phrase', 'context', 'id']

In [51]:
import re

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", flags=re.UNICODE)

def process_context(context):
    new_context = []
    for turn in context:
        new_turn = {}
        
        new_turn['user'] = re.sub(emoji_pattern, '', turn['user']).strip()
        new_turn['marusia'] = re.sub(emoji_pattern, '', turn['marusia']).strip()
        
        new_context.append(new_turn)
    return new_context

In [52]:
data['context'] = data['context'].apply(lambda x: process_context(x))

/tmp/ipykernel_989829/2551788937.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['context'] = data['context'].apply(lambda x: process_context(x))


In [53]:
data['user_replics'] = data['context'].apply(lambda x: '\n'.join([x_i['user'] for x_i in x]))

/tmp/ipykernel_989829/903242129.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['user_replics'] = data['context'].apply(lambda x: '\n'.join([x_i['user'] for x_i in x]))


In [54]:
data['user_replics'] = data['user_replics'].apply(lambda x: re.sub('\n+', '\n', x))

/tmp/ipykernel_989829/2610798981.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['user_replics'] = data['user_replics'].apply(lambda x: re.sub('\n+', '\n', x))


In [56]:
data['marusia_replics'] = data['context'].apply(lambda x: '\n'.join([x_i['marusia'] for x_i in x]))
data['marusia_replics'] = data['marusia_replics'].apply(lambda x: re.sub('\n+', '\n', x))

In [57]:
data.loc[data['marusia_replics'] == '\n', 'marusia_replics'] = 'Нет реплики'
data.loc[data['user_replics'] == '\n', 'user_replics'] = 'Нет реплики'

In [60]:
data[['marusia_replics', 'user_replics', 'phrase']].to_csv('prep_punc_test.tsv', index=False, sep='\t')

In [ ]:
!python3 readability.py --doc_path ../../marusya/prep_punc_test.tsv --column user_replics

In [ ]:
!python3 readability.py --doc_path ../../marusya/prep_punc_test.tsv --column phrase

In [74]:
csv_data = pd.read_csv('punc_test.tsv', sep='\t')

In [76]:
data_user_replics = pd.read_csv('readability_data/prep_user_replics_prep_punc_test.tsv', sep='\t')
data_phrase = pd.read_csv('readability_data/prep_phrase_prep_punc_test.tsv', sep='\t')

user_replics_cols = ['TTR', 'Количество абзацев', 'Количество слогов', 'Количество слов']
phrase_cols = ['Среднее количество слов в предложении', 'Существительных', 'Количество слогов', 'RR']
readability_parse_user_repl = data_user_replics[user_replics_cols]
readability_parse_user_repl.columns = [column + ' контекста юзера' for column in readability_parse_user_repl.columns]
readability_parse_phrase = data_phrase[phrase_cols]
readability_parse_phrase.columns = [column + ' таргетной фразы' for column in readability_parse_phrase.columns]
merged_data = pd.merge(csv_data.reset_index(), readability_parse_user_repl.reset_index(), on='index')
merged_data = pd.merge(merged_data, readability_parse_phrase.reset_index(), on='index')
feature_cols = list(readability_parse_phrase.columns) + list(readability_parse_user_repl.columns)
csv_data = merged_data.copy()
csv_data.drop(['index'], axis=1, inplace=True)

In [65]:
from utils import *

In [82]:
csv_data['context'] = csv_data['context'].apply(lambda x: eval(x))

In [84]:
csv_data['toxicity_phrase'] = csv_data['phrase'].apply(lambda x: text2toxicity(x, True))
csv_data['toxicity_user_context'] = csv_data['context'].apply(lambda x: toxicity_of_context(x))
csv_data['linguistic_acceptability'] = csv_data['phrase'].apply(lambda x: predict_acceptability(x))
csv_data['str_for_resp_qual'] = csv_data.apply(return_str_for_resp_qual, axis=1)
csv_data['resp_qual'] = csv_data['str_for_resp_qual'].apply(lambda x: predict_resp_qual(x))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [87]:
csv_data['relevance'] = csv_data['resp_qual'].apply(lambda x: x.split()[0]).astype('float64')
csv_data['specificity'] = csv_data['resp_qual'].apply(lambda x: x.split()[1]).astype('float64')
csv_data.drop(['resp_qual', 'str_for_resp_qual'], axis=1, inplace=True)
feature_cols.extend(['toxicity_phrase', 'toxicity_user_context', 'specificity', 'relevance', 'linguistic_acceptability'])
csv_data.to_csv('add_features_test.tsv', sep='\t', index=False)

In [201]:
csv_data = pd.read_csv('add_features_test.tsv', sep='\t')

In [204]:
csv_data['context'] = csv_data['context'].apply(eval)

In [202]:
csv_data.columns

Index(['phrase', 'context', 'TTR контекста юзера',
       'Количество абзацев контекста юзера',
       'Количество слогов контекста юзера', 'Количество слов контекста юзера',
       'Среднее количество слов в предложении таргетной фразы',
       'Существительных таргетной фразы', 'Количество слогов таргетной фразы',
       'RR таргетной фразы', 'toxicity_phrase', 'toxicity_user_context',
       'linguistic_acceptability', 'relevance', 'specificity'],
      dtype='object')

In [97]:
feature_cols = csv_data.columns[2:]

In [98]:
feature_cols

Index(['TTR контекста юзера', 'Количество абзацев контекста юзера',
       'Количество слогов контекста юзера', 'Количество слов контекста юзера',
       'Среднее количество слов в предложении таргетной фразы',
       'Существительных таргетной фразы', 'Количество слогов таргетной фразы',
       'RR таргетной фразы', 'toxicity_phrase', 'toxicity_user_context',
       'linguistic_acceptability', 'relevance', 'specificity'],
      dtype='object')

In [205]:
model_type = 'roberta'
add_response_token = False

if model_type == 'bert':
    csv_data['prep_context'] = csv_data['context'].apply(lambda x: preprocess_context(x, 'bert'))
    if add_response_token:
        add_token = '[RESPONSE_TOKEN]'
        csv_data['prep_phrase_context'] = csv_data['prep_context'] + add_token + '— ' + csv_data['phrase'] + ' [SEP]'
    else:
        csv_data['prep_phrase_context'] = csv_data['prep_context'] + '— ' + csv_data['phrase'] + ' [SEP]'
elif model_type == 'roberta':
    csv_data['prep_context'] = csv_data['context'].apply(lambda x: preprocess_context(x, 'roberta'))
    if add_response_token:
        add_token = '[RESPONSE_TOKEN]'
        csv_data['prep_phrase_context'] = csv_data['prep_context'] + add_token + '– ' + csv_data['phrase'] + '</s>'
    else:
        csv_data['prep_phrase_context'] = csv_data['prep_context'] + '– ' + csv_data['phrase'] + '</s>'
elif model_type == 't5':
    csv_data['prep_context'] = csv_data['context'].apply(lambda x: preprocess_context(x, 't5'))
    csv_data['prep_phrase_context'] = csv_data['prep_context'] + ' <extra_id_0> ' + '— ' + csv_data['phrase'] + ' <extra_id_0>'

In [206]:
csv_data = csv_data[['prep_phrase_context'] + list(feature_cols)]

In [208]:
import pickle

with open('scaler.pickle', 'rb') as handle:
    scaler = pickle.load(handle)

In [209]:
test_data, test_features = csv_data.values[:, 0], csv_data.values[:, 1:]

In [210]:
scaled_test_features = scaler.transform(test_features)

In [211]:
import test_dataset
import importlib
importlib.reload(test_dataset)
from test_dataset import TestDataset

In [216]:
hugg_name = 'DeepPavlov/xlm-roberta-large-en-ru'

tokenizer = AutoTokenizer.from_pretrained(hugg_name)

# special_tokens_dict = {'additional_special_tokens': ['[RESPONSE_TOKEN]']}
# tokenizer.add_special_tokens(special_tokens_dict)

config = {'MAX_LEN': 256, 'tokenizer': tokenizer, 'model': model_type}

dataset = TestDataset(test_data, 
                config=config,
                features=scaled_test_features
                )

In [134]:
from model_factory import create_model

In [135]:
!ls checkpoints/roberta_bs8_adam_gradclip_rocauctrack_bceloss_addfeat

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
'epoch=2-step=942.ckpt'


In [218]:
model_type = 'roberta'

model = create_model(model_type, 'bce', len(feature_cols))

model_path = 'model2.pt'

# model.model.resize_token_embeddings(len(tokenizer))

def read_model(model, path):
    state_dict = torch.load(path)['state_dict']
    re_state_dict = {key.replace('model.', '', 1): value for key, value in state_dict.items()}
    model.load_state_dict(re_state_dict)
    return model

new_model = read_model(model, model_path)

In [219]:
model.to('cpu')

BertClassifier(
  (model): XLMRobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(35055, 1024)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((102

In [220]:
device = torch.device('cpu' if not torch.cuda.is_available() else 'cuda:11')

model.to(device)

BertClassifier(
  (model): XLMRobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(35055, 1024)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((102

In [221]:
from tqdm import tqdm

In [232]:
def pred_probs(dataset, model):
    pred_probs = []
    for batch in tqdm(iter(dataset)):
        ids = batch['ids'].unsqueeze(0).to(device)
        attention_mask = batch['att_mask'].unsqueeze(0).to(device)
        features = batch['features'].unsqueeze(0).to(device)
        out_probs = model.forward(input_ids=ids, attention_mask=attention_mask, features=features)
        logits = torch.nn.Sigmoid()(out_probs[0])
        print(logits)
#         pred_label = torch.where(out_probs > 0.5, 1, 0).item()
        pred_probs.append(logits[1].item())
        ids = ids.to('cpu')
        attention_mask = attention_mask.to('cpu')
        features = features.to('cpu')
        del ids
        del attention_mask
        del features
    return pred_probs

In [ ]:
test_pred_probs = pred_probs(dataset, model)

In [167]:
!mkdir submissions

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [198]:
submission_data = pd.DataFrame({'id': range(len(test_pred_probs)), 'target': test_pred_probs})
submission_data.to_csv('submissions/submission_5.csv', sep=';', index=False)